In [1]:
from luna.data_processing.utils import *
from luna.models import xgb_reg
from luna.hyper_opt import OptunaTuner

d:\anaconda3\envs\fafa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = load_data('./example_data/regression/train.csv', 'csv')
df_train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [3]:
target = 'Listening_Time_minutes'

In [4]:
df_train = preprocess_data(df_train, target)

In [5]:
X_train, X_val, y_train, y_val = split_data(df_train, target)

In [6]:
from xgboost import XGBRegressor
# 定义参数搜索空间
param_space = {
    'learning_rate': ('float', 0.01, 0.3),
    'n_estimators': ('int', 100, 1000),
    'booster': ('categorical', ['gbtree', 'gblinear'])
}

# 创建调参器
tuner = OptunaTuner(
    param_space=param_space,
    n_trials=5,
    direction='minimize',
    metric_name='rmse'
)

# 执行优化
results = tuner.optimize(XGBRegressor, X_train, y_train, X_val, y_val)

# 获取最优参数
best_params = results['best_params']

[I 2025-05-06 19:03:24,486] A new study created in memory with name: no-name-bb2e1a92-ef1a-40c2-9b71-7965ded51991
[I 2025-05-06 19:04:22,213] Trial 0 finished with value: 13.360316334223675 and parameters: {'learning_rate': 0.2796641198488743, 'n_estimators': 975, 'booster': 'gblinear'}. Best is trial 0 with value: 13.360316334223675.
[I 2025-05-06 19:04:51,286] Trial 1 finished with value: 13.360316445649833 and parameters: {'learning_rate': 0.21652853849492282, 'n_estimators': 478, 'booster': 'gblinear'}. Best is trial 0 with value: 13.360316334223675.
[I 2025-05-06 19:05:23,993] Trial 2 finished with value: 12.994690656644055 and parameters: {'learning_rate': 0.08471421181902708, 'n_estimators': 407, 'booster': 'gbtree'}. Best is trial 2 with value: 12.994690656644055.
[I 2025-05-06 19:05:56,918] Trial 3 finished with value: 13.36031631242089 and parameters: {'learning_rate': 0.2838480309170139, 'n_estimators': 750, 'booster': 'gblinear'}. Best is trial 2 with value: 12.994690656644

In [7]:
xgb_reg = xgb_reg(best_params)
xgb_reg.fit(X=X_train, y=y_train)

In [8]:
xgb_reg.evaluate(X_val, y_val)

目标值范围: [0.000, 119.970]
目标值统计:
均值: 45.396
标准差: 27.126
中位数: 43.362



{'rmse': 12.994690656644055, 'r2': 0.7705141178733519}